# GymPy

Note:
Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint as pprint

# Import API key
from api_keys import g_key

In [5]:
# Part 1 data file
master_data_path = "Output/master_data.csv"
lat_lng_path = "Output/postcode_df.csv"

# Read the metadata 
master_data_df = pd.read_csv(master_data_path)
lat_lng_df = pd.read_csv(lat_lng_path)

In [6]:
master_data_df.shape

(2641, 21)

In [7]:
lat_lng_df.shape

(2641, 6)

In [8]:
master_data_df.columns

Index(['Postcode', 'House', 'Semi-detached', 'Apartment', 'Other dwelling',
       'No bedrooms (studios, etc)', '1 bedroom', '2 bedrooms', '3 bedrooms',
       '4 bedrooms', '5 bedrooms', '6 or more bedrooms',
       'Couple with no children', 'Couple with children',
       'One parent with children', 'Other family', 'Sole person household',
       'Group household', 'People with Mental Health Conditions',
       'Total People', 'State'],
      dtype='object')

In [9]:
master_data_df.head(3)

,Postcode,House,Semi-detached,Apartment,Other dwelling,"No bedrooms (studios, etc)",1 bedroom,2 bedrooms,3 bedrooms,4 bedrooms,...,6 or more bedrooms,Couple with no children,Couple with children,One parent with children,Other family,Sole person household,Group household,People with Mental Health Conditions,Total People,State
0,800,33,6,5876,4,172,1856,3749,2805,168,...,6,2874,1181,395,134,3420,910,344,7149,7: Northern Territory
1,810,14828,2642,5649,85,207,1768,8552,9360,5484,...,311,6553,8881,2893,425,6996,1737,1971,34330,7: Northern Territory
2,812,9601,1127,1549,134,67,448,3389,6590,2353,...,95,3237,4486,2032,204,3249,592,1076,18634,7: Northern Territory


In [10]:
lat_lng_df.head()

,Unnamed: 0,Postcode,State,Latitude,Longitude,Remoteness Area
0,0,800,NSW,-33.858556,151.180095,1.0
1,1,810,NSW,-33.866044,151.133865,1.0
2,2,812,NSW,-33.866044,151.133865,1.0
3,3,820,NSW,-33.897219,151.150850,1.0
4,4,822,NSW,-33.897219,151.150850,1.0


In [11]:
master_data_with_coords_df = pd.merge(master_data_df, lat_lng_df, how="left", on=["Postcode", "Postcode"])

master_data_with_coords_df.head()                                           

,Postcode,House,Semi-detached,Apartment,Other dwelling,"No bedrooms (studios, etc)",1 bedroom,2 bedrooms,3 bedrooms,4 bedrooms,...,Sole person household,Group household,People with Mental Health Conditions,Total People,State_x,Unnamed: 0,State_y,Latitude,Longitude,Remoteness Area
0,800,33,6,5876,4,172,1856,3749,2805,168,...,3420,910,344,7149,7: Northern Territory,0,NSW,-33.858556,151.180095,1.0
1,810,14828,2642,5649,85,207,1768,8552,9360,5484,...,6996,1737,1971,34330,7: Northern Territory,1,NSW,-33.866044,151.133865,1.0
2,812,9601,1127,1549,134,67,448,3389,6590,2353,...,3249,592,1076,18634,7: Northern Territory,2,NSW,-33.866044,151.133865,1.0
3,820,4926,3337,6733,104,167,1463,7566,7580,2372,...,5987,1843,1112,19920,7: Northern Territory,3,NSW,-33.897219,151.150850,1.0
4,822,9415,655,168,348,216,873,3083,4392,1811,...,1921,268,755,25304,7: Northern Territory,4,NSW,-33.897219,151.150850,1.0


In [12]:
master_data_with_coords_df = master_data_with_coords_df.drop('State_y', axis=1)
master_data_with_coords_df = master_data_with_coords_df.rename(columns={"State_x": "State"})

In [13]:
master_data_with_coords_df.shape

(2641, 25)

In [14]:
master_data_with_coords_df.head()

,Postcode,House,Semi-detached,Apartment,Other dwelling,"No bedrooms (studios, etc)",1 bedroom,2 bedrooms,3 bedrooms,4 bedrooms,...,Other family,Sole person household,Group household,People with Mental Health Conditions,Total People,State,Unnamed: 0,Latitude,Longitude,Remoteness Area
0,800,33,6,5876,4,172,1856,3749,2805,168,...,134,3420,910,344,7149,7: Northern Territory,0,-33.858556,151.180095,1.0
1,810,14828,2642,5649,85,207,1768,8552,9360,5484,...,425,6996,1737,1971,34330,7: Northern Territory,1,-33.866044,151.133865,1.0
2,812,9601,1127,1549,134,67,448,3389,6590,2353,...,204,3249,592,1076,18634,7: Northern Territory,2,-33.866044,151.133865,1.0
3,820,4926,3337,6733,104,167,1463,7566,7580,2372,...,220,5987,1843,1112,19920,7: Northern Territory,3,-33.897219,151.150850,1.0
4,822,9415,655,168,348,216,873,3083,4392,1811,...,404,1921,268,755,25304,7: Northern Territory,4,-33.897219,151.150850,1.0


In [15]:
pop_greater_1000_coord_df = master_data_with_coords_df.loc[master_data_with_coords_df["Total People"] >= 10000]

In [16]:
pop_greater_1000_coord_df.shape

(852, 25)

In [17]:
wa_pop_greater_1000_df = pop_greater_1000_coord_df.loc[pop_greater_1000_coord_df["State"] == "5: Western Australia"]

In [18]:
wa_pop_greater_1000_df.shape

(90, 25)

In [19]:
# Store WA Postcode by WA greater than 10,000 population 
wa_pg1000_postcode = wa_pop_greater_1000_df["Postcode"].astype(int)

#Need to add # of gyms
#location_hum = cities_metadata_df.dropna()
#loc_humidity = cities_metadata_df["Humidity"].astype(float)

In [20]:
wa_pg1000_postcode

2140    6000
2142    6004
2146    6008
2147    6009
2148    6010
        ... 
2412    6450
2458    6530
2505    6714
2510    6722
2511    6725
Name: Postcode, Length: 90, dtype: int32

In [21]:
wa_pg1000_locations = wa_pop_greater_1000_df[["Latitude", "Longitude"]].astype(float)

In [22]:
wa_pg1000_locations

,Latitude,Longitude
2140,-32.786850,148.735871
2142,-32.740876,148.561505
2146,-32.584028,148.266655
2147,-32.584028,148.266655
2148,-33.035496,148.168190
...,...,...
2412,-37.817798,145.174489
2458,-37.921894,145.389200
2505,-38.181608,144.427419
2510,-38.209921,144.572989


In [23]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [24]:
#Figure(layout=FigureLayo# Customise the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)

In [26]:
marker_layer = gmaps.symbol_layer(wa_pg1000_locations)
fig_gyms = gmaps.figure(center = (0, 30) ,zoom_level = 2)
fig_gyms.add_layer(marker_layer)
fig_gyms

Figure(layout=FigureLayout(height='420px'))